In [ ]:
%cd ..

In [ ]:
import requests
import requests.exceptions
import time

# Work with API

## Projects

### List projects

Here we get default projects list. It contains just one - default - project.

In [ ]:
projects = requests.get('http://0.0.0.0:8080/projects')
projects.json() # there is default (blank) project 

### Create new project

Let's create new project and work with it

In [ ]:
create_project = requests.post('http://0.0.0.0:8080/projects', data={'project': 'Demo'})
create_project.json()

and now get projects list again

In [ ]:
requests.get('http://0.0.0.0:8080/projects').json()

As you can see, new project **Demo** is created: id = 2, mlflowUri = http://0.0.0.0:5001

### Run project
To communicate with project **Demo** run it.

In [ ]:
run_project = requests.put('http://0.0.0.0:8080/projects/2/run')
run_project.status_code

### Run examples - machine learning scripts

As projects was run you can work with it.
You can observe scripts before running.

In [ ]:
# Wait while project's tracking server starting (MLflow tracking server take some time to start)

while True:
    print('Try to connect...')
    
    response = requests.get('http://0.0.0.0:8080/projects/2/ping')
    
    if response.status_code == 200:
        print('connected!')
        break
    
    time.sleep(1)


In [ ]:
%%bash

# Set tracking server URI - from project **Demo**
export MLFLOW_TRACKING_URI=http://0.0.0.0:5001

# Run examples. Consider each script as some ML-project entrypoint script or stage of ML-pipeline

# example1 
python example1.py

# example2 with default parameters
python example2.py

# example2 with parameters
python example2.py --C 0.1 --solver newton-cg

# example3 with default parameters
python example3.py

# example3 with parameters
python example3.py --C 0.1 --kernel linear

## Experiments

In previous section:

* new project was created and run;
* some scripts was executed.

So now we have several experiments and runs.

### List experiments

In [ ]:
experiments = requests.get('http://0.0.0.0:8080/experiments?project_id=2')
experiments.json()

## Runs

### List runs

Get runs for experiment **IrisLogreg** of project **Demo**

In [ ]:
runs = requests.get('http://0.0.0.0:8080/runs?project_id=2&experiment_id=1')
runs.json()

## Registered Models

In scripts `example2.py` and `example3.py` registred models are created:   
when models are built records about them are added into Model Registry using **MLflow** API

### List registered models

In [ ]:
registered_models = requests.get('http://0.0.0.0:8080/registered-models?project_id=2')
registered_models.json()

### List model versions
Each model has several versions.  
Get all versions of all registered models in project **Demo**

In [ ]:
# by defualt - all model version for project
model_versions = requests.get('http://0.0.0.0:8080/model-versions?project_id=2')
model_versions.json()

also you can get versions for specific registered model by *model_id*:

In [ ]:
iris_logreg_model_versions = requests.get(
    'http://0.0.0.0:8080/model-versions?project_id=2&model_id=IrisLogregModel')
iris_logreg_model_versions.json()

## Deployments
There is some registered models and its versions.   
Let's deploy one model version.

### List deployments
Make sure no deployments by default

In [ ]:
deployments = requests.get('http://0.0.0.0:8080/deployments')
deployments.json()

### Create deployment

In [ ]:
create_deployment = requests.post(
    'http://0.0.0.0:8080/deployments?project_id=2&model_id=IrisLogregModel&version=2&type=local',
)
create_deployment.json()

list deployments again

In [ ]:
requests.get('http://0.0.0.0:8080/deployments').json()

## Predict
If deployment is running you can predict data.

In [ ]:
# Wait while model server starting (MLflow model server take some time to start)

while True:
    print('Try to connect...')
    
    response = requests.get('http://0.0.0.0:8080/deployments/1/ping')
    
    if response.status_code == 200:
        print('connected!')
        break
    
    time.sleep(1)

In [ ]:
predict = requests.post('http://0.0.0.0:8080/deployments/1/predict', data={'data': '[[1,2,3,4], [5,6,7,8]]'})
predict.json()

## Stop deployment

In [ ]:
stop_deployment = requests.put('http://0.0.0.0:8080/deployments/1/stop')
stop_deployment.json()

In [ ]:
# list deployments again
requests.get('http://0.0.0.0:8080/deployments').json()

**Notice changes:** *status = stopped* and *host = port = None*

## Delete deployment

In [ ]:
delete_deployment = requests.delete('http://0.0.0.0:8080/deployments/1')
delete_deployment.json()

list deployments again:

In [ ]:
requests.get('http://0.0.0.0:8080/deployments').json()

deployment is deleted!